Caixeiro com gasolina infinita
========================================



## Introdução



Muito semelhante ao experimento A.06, contudo, ao invez de ser um problema de minimização, este é de maximização, já que o caixeiro viajante é contra as políticas ambientais, e sembre busca a maior distãncia entre as cidades, seguindo um caminho que consome muita gasolina. 



## Objetivo



Encontrar o caminho de *maior* distância no problema do caixeiro viajante e mostrar ele de forma gráfica.

**Considerações do experimento**: Considere um número $n\geq 7$ de coordenadas $(x,y)$ de cidades e que o caixeiro tenha combustível infinito. Você pode gerar as coordenadas de forma aleatória ou simplesmente usar as coordenadas que desejar. O caixeiro só anda em linha reta e apenas entre duas cidades. O caixeiro começa e termina seu trajeto na mesma cidade e, fora a cidade inicial, ele não visita nenhuma outra cidade mais de uma vez.



## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import random
from itertools import permutations

import graphviz
from collections import deque

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import selecao_torneio_max # oq mudou?
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

from funcoes import funcao_objetivo_cv_gasolina_inf as funcao_objetivo_individuo # Não entendi

from time import perf_counter as pf

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca 
# Os seguintes valores são exatamente os mesmos usados no experimento do caixeiro viajante. 
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 7 # Apenas o número de cidades que muda para cumprir com os objetivos do experimento
CIDADES = cria_cidades(NUMERO_DE_CIDADES)


In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_selecao(populacao, fitness):
    return selecao_torneio_max(populacao, fitness, NUM_COMBATENTES_NO_TORNEIO)

Abaixo temos o mesmo código usado no experimento A.06

In [4]:
# Busca por algoritmo genético

start_algoritmo = pf()

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = -float("inf")

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    if maior_fitness > melhor_fitness_ja_visto:        
        posicao = fitness.index(maior_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = maior_fitness    
    
fim_algoritmo = pf()

Para comprovar que o método usando de busca por algoritmo genético resulta no melhor resultado possível, então, realizamos uma busca por exaustão para ter a certeza de que os resultados são condizentes, já que todos os caminhos serão analisados.

In [5]:
# Busca testando todas as permutações

melhor_fitness_exaustao = -float("inf")

inicio_all = pf()

# testando todas as permutações possíveis
for caminho in permutations(list(CIDADES.keys())):
    caminho = list(caminho)
    distancia = funcao_objetivo_individuo(caminho, CIDADES)
    if distancia > melhor_fitness_exaustao:
        melhor_fitness_exaustao = distancia
        melhor_resposta_exaustao = caminho

fim_all = pf()

In [6]:
# Checando os resultados

print()
print("Melhor caminho das cidades obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com a distância de:", melhor_fitness_ja_visto, f'em {fim_algoritmo-start_algoritmo} segundos')

print()
print("Melhor caminho das cidades obtido por busca exaustiva:")
print(melhor_resposta_exaustao, "com a distância de:", melhor_fitness_exaustao, f'em {fim_all-inicio_all} segundos')


Melhor caminho das cidades obtido por algoritmos genéticos:
['Cidade 6', 'Cidade 4', 'Cidade 3', 'Cidade 5', 'Cidade 1', 'Cidade 2', 'Cidade 0'] com a distância de: 4.159364188813353 em 0.7728840999999997 segundos

Melhor caminho das cidades obtido por busca exaustiva:
['Cidade 2', 'Cidade 5', 'Cidade 1', 'Cidade 3', 'Cidade 4', 'Cidade 6', 'Cidade 0'] com a distância de: 4.159364188813353 em 0.0382100000000003 segundos


É retornado o caminho das cidades, e conforme os objetivos, nenhuma cidade foi repetida. Além disso, por meio da função objetivo temos o desejado, junto da seleção, esta que busca uma maximização dada pela `selecao_torneio_max`. Então é possível visualizar que os resultados são condizentes, haja vista que a sistancia de ambos os métodos são iguais e junto a eles temos o dempo que o código demorou para processar. Uma observação importante é que de forma um pouco contra intuitiva, o método de exaustão foi mais rápido do que o de algoritmo genético, e isso se dá pela quantidade de cidades do problema, onde o número de possíbilidades vai aumentando de forma fatorial, assim, também foi realizado usando `NUMERO_DE_CIDADES` = 10 e o tempo obtido foi cerca de 100 segundos.

In [7]:
g = graphviz.Digraph(format='svg', graph_attr={'rankdir': 'LR'}, comment='Maior caminho possível')

for node in melhor_individuo_ja_visto:
    g.node(node, node)

individuo_copy = deque(melhor_individuo_ja_visto)
individuo_copy.rotate(-1)
    
for ini, che in zip(melhor_individuo_ja_visto, individuo_copy):
    g.edge(ini, che)

g

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

Assim mostrando de forma gráfica por meio de um grafo, como foi o caminho percorrido.

## Observações

Esse é o segundo experimento que estou realizando da lista de experimentos de **Algoritmos Géneticos**.

## Conclusão



A

## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

